# Partially Signed Bitcoin Transaction

In [ ]:
%load_ext autoreload
%autoreload 2

## Init Bitcoin RPC client

In [ ]:
import os
import bitcoin
from rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
rpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
rpc.getblockcount()

## Create/Load wallet

In [ ]:
rpc.createwallet('miner')
# rpc.loadwallet('miner')

In [ ]:
rpc.getbalance()

## Mine coins

In [ ]:
import bitcoin.core as bc

In [ ]:
minerAddr = rpc.getnewaddress("coinbase")
minerAddr

In [ ]:
# coinbase transactions are locked for 100 blocks
_ = list(rpc.generatetoaddress(101, minerAddr))

In [ ]:
rpc.getbalance()

In [ ]:
unspent = rpc.listunspent()
unspent

In [ ]:
# coinbase transaction
r = rpc.getrawtransaction(unspent[0]['outpoint'].hash, verbose=True)
r

In [ ]:
# Coinbase transaction id
bc.b2lx(r['tx'].GetTxid())

## Fund Ali and Bob

In [ ]:
aliAddr = rpc.getnewaddress("ali")
aliAddr

In [ ]:
bobAddr = rpc.getnewaddress("bob")
bobAddr

In [ ]:
fundTxId = rpc.sendmany("", {aliAddr: 10*bc.COIN, bobAddr: 11*bc.COIN}, comment="Funding Ali and Bob")

In [ ]:
rpc.getrawtransaction(fundTxId, verbose=True)

In [ ]:
# confirm transactions
_ = list(rpc.generatetoaddress(6, minerAddr))

In [ ]:
rpc.listunspent(addrs=[aliAddr])

In [ ]:
rpc.listunspent(addrs=[bobAddr])

## Create 2-of-2 Multisig

In [ ]:
import bitcoin.core.script as bs

### Generate keys for multisig

In [ ]:
# h = hashlib.sha256(b'correct horse battery staple').digest()
# seckey = CBitcoinSecret.from_secret_bytes(h)

In [ ]:
aliMsigAddr = rpc.getnewaddress('ali multisig')
bobMsigAddr = rpc.getnewaddress('bob multisig')

In [ ]:
aliMsigSecret = rpc.dumpprivkey(aliMsigAddr)
bobMsigSecret = rpc.dumpprivkey(bobMsigAddr)

### Get inputs
aka ali's and bob's funding tx outputs

In [ ]:
aliTxin = bc.CTxIn(rpc.listunspent(addrs=[aliAddr])[0]['outpoint'])
aliTxin

In [ ]:
bobTxin = bc.CTxIn(rpc.listunspent(addrs=[bobAddr])[0]['outpoint'])
bobTxin

### Construct multisig

In [ ]:
# redeem script
msigScript = bs.CScript([
        bs.OP_2,
        aliMsigSecret.pub,
        bobMsigSecret.pub,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])
msigScript

### Create output

In [ ]:
scriptPubKey = msigScript.to_p2sh_scriptPubKey()
scriptPubKey

In [ ]:
fee = 0.001 
txout = bc.CTxOut((10+11-fee)*bc.COIN, scriptPubKey)
txout

### Create transaction

In [ ]:
tx = bc.CMutableTransaction([aliTxin, bobTxin], [txout])
tx

### Sign transaction

In [ ]:
# sign by Ali
sighash = bs.SignatureHash(aliAddr.to_redeemScript(),
                           tx,
                           inIdx=0,
                           hashtype=bs.SIGHASH_ALL,
                           amount=10*bc.COIN,
                           sigversion=bs.SIGVERSION_WITNESS_V0)

secret = rpc.dumpprivkey(aliAddr)
aliSignature = secret.sign(sighash) + bytes([bs.SIGHASH_ALL])

aliWitness = [aliSignature, secret.pub]
aliWitness = bc.CTxInWitness(bs.CScriptWitness(aliWitness))
aliWitness

In [ ]:
# sign by Bob
sighash = bs.SignatureHash(bobAddr.to_redeemScript(),
                           tx,
                           inIdx=1,
                           hashtype=bs.SIGHASH_ALL,
                           amount=11*bc.COIN,
                           sigversion=bs.SIGVERSION_WITNESS_V0)

secret = rpc.dumpprivkey(bobAddr)
bobSignature = secret.sign(sighash) + bytes([bs.SIGHASH_ALL])

bobWitness = [bobSignature, secret.pub]
bobWitness = bc.CTxInWitness(bs.CScriptWitness(bobWitness))
bobWitness

In [ ]:
tx.wit = bc.CTxWitness([aliWitness, bobWitness])
tx

### Submit transaction

In [ ]:
msigTxid = rpc.sendrawtransaction(tx)

In [ ]:
_ = list(rpc.generatetoaddress(6, minerAddr))

In [ ]:
rpc.getrawtransaction(msigTxid, verbose=True)